In [1]:
import  torch
from    torch_geometric.data import Data

/Users/shervin/opt/anaconda3/envs/AIDAVA/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.chdir('..') # set directoroy to AIDAVA-KGE-Framework (run once)

In [3]:
import  pickle 

from    src.gcn import GCN

import  torch.nn.functional as F
import  tqdm

### Load and prepare data

In [4]:
data_path   = './data/m2skg/'
filename    = 'assertion_train.pkl'

node_embed_dim  = 10

In [5]:
with open(data_path+filename,'rb') as f:
    data = pickle.load(f)

In [8]:
entities= {e : i for i,e in enumerate( list(set([s for s,t in data])))}
types   = {t : i for i,t in enumerate( list(set([t for s,t in data])))}

In [9]:
types # toy data from one of SPHN classes

{'LabTestEvent': 0,
 'DrugPrescription': 1,
 'OrganSupport': 2,
 'Code': 3,
 'AdministrativeCase': 4,
 'SubjectPseudoIdentifier': 5}

### TransE for Node Prediction (demo)

In [10]:
from    src.TransE import TransE

In [11]:
device = torch.device('cpu')

In [12]:
trans_e = TransE(num_entities = len(entities),
       num_relations = 1,
       device = device)

In [13]:
triples = []

r = 0 # one relation, i.e. rdfs:type 

for i,(entity,type) in enumerate(data): 

    triples.append( ( entities[entity], 0 , types[type] ) )

In [14]:
triples[0] # head, relation, tail

(358, 0, 4)

In [15]:
trans_e._train(triples,num_epochs=20);

/Users/shervin/Documents/AIDAVA/aidava-kge-framework/src/loaders.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.edges_index[idx][0]), self.edges_index[idx][1],   torch.tensor(self.edges_index[idx][2]), torch.tensor(neg_sample[0][0]) , torch.tensor(neg_sample[1]) , torch.tensor(neg_sample[0][1])


epoch 0,	 train loss 1.09
epoch 2,	 train loss 1.11
epoch 4,	 train loss 1.07
epoch 6,	 train loss 1.04
epoch 8,	 train loss 1.04
epoch 10,	 train loss 1.07
epoch 12,	 train loss 1.04
epoch 14,	 train loss 1.03
epoch 16,	 train loss 1.02
epoch 18,	 train loss 1.00


In [16]:
trans_e._eval(triples)

hits@1 0.06 hits@10 0.20  MR 43.72 MRR  0.12
